In [1]:
let o f g = fun x -> g(f(x))

val o : ('a -> 'b) -> ('b -> 'c) -> 'a -> 'c = <fun>


In [2]:
let rec repeat f n x = match n with
| 0 -> f x
| n -> repeat (o f f) (n-1) x

val repeat : ('a -> 'a) -> int -> 'a -> 'a = <fun>


In [3]:
let succ x = x + 1


val succ : int -> int = <fun>


In [4]:
repeat succ 1 5


- : int = 7


In [24]:
let (--) i j =
  let rec from i j l =
    if i>j then l
    else from i (j-1) (j::l)
    in from i j []
let rec filter f = function
  | [] -> []
  | h::t -> if f h then h::(filter f t) else filter f t
let rec map f = function
  | [] -> []
  | h::t -> (f h)::(map f t)
let rec fold_left op acc = function
  | []   -> acc
  | h :: t -> fold_left op (op acc h) t
let rec length = function
  | [] -> 0
  | x :: xs -> 1 + length xs 

val ( -- ) : int -> int -> int list = <fun>


val filter : ('a -> bool) -> 'a list -> 'a list = <fun>


val map : ('a -> 'b) -> 'a list -> 'b list = <fun>


val fold_left : ('a -> 'b -> 'a) -> 'a -> 'b list -> 'a = <fun>


val length : 'a list -> int = <fun>


In [25]:
let sum_cube_odd n = fold_left (fun a x -> a + (x * x * x)) 0 (filter (fun x -> x mod 2 == 0) (0--n))


val sum_cube_odd : int -> int = <fun>


classic example punned from PL literature

improvements:

or filter in the folded function to get rid of a pass

or construct the stream, filter it on construction, and apply the fold in one pass


Haskell list fusion (total maps) example
(todo extend with stream generators + folds)


```Haskell 
data Term a b where
    Map :: (a -> b) -> Term a b
    Filter :: (a -> Maybe a) -> Term a (Maybe a)
    
fuseMap :: Term a b -> Term b c -> Term a c
fuseMap (Map f ) (Map g) = Map $ g . f

filterFuse :: Term a (Maybe a) -> Term a b ->  Term a (Maybe b)
filterFuse (Filter p) (Map f ) = Map (\x -> p x >>= Just . f)

fuseFilter :: Term a b -> Term b (Maybe b) -> Term a (Maybe b)
fuseFilter (Map f) (Filter p) = Map $ p . f

-- infix syntax
(||) = fuseMap
(>|) = filterFuse
(|>) = fuseFilter

filter p = Filter $ \x -> case p x of
                            True -> Just x
                            _ -> Nothing

map :: (a -> b) -> Term a b
map = Map

run :: Term a b -> [a] -> [b]
run (Map f) xs = fmap f xs

-- example
run (map (+1) || map (^2) |> filter even) [1..6] ==> [Just 4,Nothing,Just 16,Nothing,Just 36,Nothing]

(map (+1) || map (^2) |> filter even) ==> Map (\x ->  case (even ((x+1)^2)) of
                                                        True -> Just x
                                                        _ -> Nothing )

```


--see fold and fusion laws

https://www.cs.ox.ac.uk/people/ralf.hinze/publications/IFL10.pdf

^^ fusion

https://www.ccs.neu.edu/home/amal/course/7480-s12/deforestation-notes.pdf

see also 'deforestation'

http://research.nii.ac.jp/~hu/pub/fac12.pdf

map reduce should have something like this in its core calculus


In [26]:
sum_cube_odd 10

- : int = 1800


In [30]:
let exists p xs = length (filter p xs) > 1

val exists : ('a -> bool) -> 'a list -> bool = <fun>


treemap (haskell code golf)

```Haskell
data Tree a = Leaf | Node a (Tree a) (Tree a) deriving (Functor, Show)

add1 = fmap (+1)
```

```Haskell
curry :: ((a,b) -> c) -> a -> b -> c
curry f = (\a -> \b -> f (a,b))

uncurry :: (a -> b -> c) -> (a,b) -> c
uncurry f = (\p -> f (fst p) (snd p))
```

tree fold (via catamorphism)

```Haskell
data TreeF a b = Leaf a| Node a b b deriving Functor

data Fix f = In {out :: (f (Fix f))}

type Tree a = Fix (TreeF a)

type Algebra f a = f a -> a

-- catamorphism (fold for F-algebras)
cata :: Functor f => Algebra f a -> (Fix f) -> a
cata alg = alg . fmap (cata alg) . out

-- END setup

-- Start example

-- example of Tree
val :: Tree Int
val = (In (Node 9 (In (Leaf 2)) (In (Leaf 8))))

--smart constructors   
leaf :: a -> Tree a 
leaf = In . Leaf
node :: a -> Tree a -> Tree a -> Tree a
node v l r = In (Node v l r) 

-- example of Tree using smart constructors
val2 :: Tree Int
val2 = node 9 (leaf 3) (leaf 9)


-- an algebra to fold over the tree with 
--note: sub terms are already computed in match
--note: no recursive calls to sumTreeAlg
sumTreeAlg :: Algebra (TreeF Int) Int
sumTreeAlg (Leaf n) = n 
sumTreeAlg (Node n l r) = n + l + r


-- use catamorphism to fold the algebra over a term
sumTree = cata sumTreeAlg

-- usage
sumTree val2 ==> 21
```

Valid Matrix (using the typechecker to determine if the matrix is valid):r

```Haskell
-- type level natural numbers
data Z = Z

data S a = S a

type Zero = Z
type One = S Zero
type Two = S One
type Three = S Two
type Four = S Three

-- size indexed lists
data List s a where
    Nil :: List Z a
    Cons :: a -> List s a -> List (S s) a

-- s row, s' col
-- size indexed matrix
data Mat s s' a where
    Empty :: Mat Z Z a
    R :: List s (List s' a) -> Mat s s' a

r1 = Cons 1 $ Cons 2 $ Cons 3 Nil
r2 = Cons 4 $ Cons 5 $ Cons 6 Nil


mat :: Mat Two Three Int
mat = R $ Cons r1 $ Cons r2 Nil
```

Same as above but with using GHC type literals and unleashing the compiler extensions

```Haskell
{-# LANGUAGE DataKinds           #-}
{-# LANGUAGE DeriveFunctor       #-}
{-# LANGUAGE GADTs               #-}
{-# LANGUAGE KindSignatures      #-}
{-# LANGUAGE RankNTypes          #-}
{-# LANGUAGE ScopedTypeVariables #-}
{-# LANGUAGE TypeApplications    #-}
{-# LANGUAGE TypeOperators       #-}
module MadLit where
import GHC.TypeNats
import Prelude hiding (Nat, length)
import           Data.Proxy


data List a = Nil | Cons a (List a) deriving Functor

length Nil = 0
length (Cons _ xs) = 1 + length xs

-- Indexed List
data SafeList (n :: Nat) a = MkList { getList :: List a}

mkList :: forall n a. KnownNat n => List a -> SafeList n a -- could use Maybe to fail gracefulls
mkList xs | length xs == fromIntegral (natVal (Proxy @n)) = MkList xs
mkList _ = undefined


test :: SafeList 3 Int
test = mkList (Cons 1 (Cons 2 (Cons 3 Nil)))

-- Indexed Matrix 

data SafeMat (n :: Nat) (m :: Nat) a = MkMat {getMat :: List (List a)}

mkSafeMat :: forall n m a. (KnownNat n, KnownNat m) => SafeList n (SafeList m a) -> SafeMat n m a
mkSafeMat mat = MkMat $ fmap getList (getList mat)


sr1 :: SafeList 3 Int
sr1 = mkList (Cons 1 ( Cons 2 ( Cons 3 Nil)))
sr2 :: SafeList 3 Int
sr2 = mkList ( Cons 4 ( Cons 5 ( Cons 6 Nil)))

mat :: SafeMat 2 3 Int
mat = mkSafeMat ( mkList ( Cons sr1 ( Cons sr2 Nil)))
```

Or using Dependent Types (Agda)

```Agda
module Mat where

data ℕ : Set where
  Z : ℕ
  S : ℕ -> ℕ

{-# BUILTIN NATURAL ℕ #-}

infixr 20 _,_
data List (A : Set) : ℕ -> Set where
  Nil : List A 0
  _,_ : {n : ℕ} -> A -> List A n -> List A (S n)

data Mat (A : Set) : ℕ -> ℕ -> Set where
  MkMat : {n m : ℕ} -> List (List A m) n -> Mat A n m

_ : Mat ℕ 2 3
_ = MkMat ((1 , 2 , 3 , Nil) ,
           (4 , 5 , 6 , Nil) , Nil)
```

Or Refinement types (Liquid Haskell, GHC plugin)

https://liquid.kosmikus.org/01-intro.html

```Haskell
{-@ measure len @-}
len :: [a] -> Int
len [] = 0
len (x : xs) = 1 + len xs

rowtest :: [[a]] -> Int -> Bool
rowtest [] n = True
rowtest (x : xs) n = len x == n && rowtest xs n 

{-@ type MList a =  {v:[[a]] |  len v > 0 && len (head v) > 0 } @-} -- && rowtest v (len v) } @-}

{-@ data Mat a = Mat (MList a) @-}
data Mat a = Mat [[a]]

m = Mat [[1]] -- typechecks
m' = Mat [[]] -- does not typecheck!
```